In [61]:
from datasets import load_dataset

unreason_ds = load_dataset("BAAI/IndustryInstruction_Health-Medicine",cache_dir="./data/un_reason/",split="train[:10]")

In [62]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [63]:
unreason_ds

Dataset({
    features: ['instruction', 'id', 'conversations', 'lang'],
    num_rows: 10
})

In [64]:
unreason_ds[0]

{'instruction': '',
 'id': 'industry-instruction-edu',
 'conversations': [{'from': 'human',
   'value': '问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关'},
  {'from': 'gpt',
   'value': '答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。'},
  {'from': 'human', 'value': '问：现在这个囊肿就已经压迫脑干了，如果以后再长大把脑干都压上了以后手术是不是就很难很危险了'},
  {'from': 'gpt',
   'value': '答：如果囊肿已经压迫了脑干，那么情况确实比较复杂和危险。手术可能会变得更具风险，因为脑干是一个重要的神经结构，手术过程中需要小心以避免额外的损伤。\n\n不过，你的具体情况需要医生进行详细的评估和决策。专业医生将会考虑多个因素，包括囊肿的位置、大小、对周围结构的影响以及你的整体健康状况等等。他们会权衡手术的利弊，并与你详细讨论手术风险和可能的好处。\n\n最重要的是，请务必在专业医生的指导下进行进一步的治疗和管理。他们能够为你提供最佳的个性化建议，并指导你选择最适合你情况的治疗方案。'}],
 'lang': 'zh'}

In [65]:
# 非推理数据集的处理，原始数据集是sharegpt格式，先处理成chatml格式
from unsloth import standardize_sharegpt
unreason_ds = standardize_sharegpt(unreason_ds)
unreason_ds

num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.


Dataset({
    features: ['instruction', 'id', 'conversations', 'lang'],
    num_rows: 10
})

In [66]:
unreason_ds[0]

{'instruction': '',
 'id': 'industry-instruction-edu',
 'conversations': [{'content': '问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关',
   'role': 'user'},
  {'content': '答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。',
   'role': 'assistant'},
  {'content': '问：现在这个囊肿就已经压迫脑干了，如果以后再长大把脑干都压上了以后手术是不是就很难很危险了', 'role': 'user'},
  {'content': '答：如果囊肿已经压迫了脑干，那么情况确实比较复杂和危险。手术可能会变得更具风险，因为脑干是一个重要的神经结构，手术过程中需要小心以避免额外的损伤。\n\n不过，你的具体情况需要医生进行详细的评估和决策。专业医生将会考虑多个因素，包括囊肿的位置、大小、对周围结构的影响以及你的整体健康状况等等。他们会权衡手术的利弊，并与你详细讨论手术风险和可能的好处。\n\n最重要的是，请务必在专业医生的指导下进行进一步的治疗和管理。他们能够为你提供最佳的个性化建议，并指导你选择最适合你情况的治疗方案。',
   'role': 'assistant'}],
 'lang': 'zh'}

In [71]:
from datasets import Dataset
import itertools

def split_conversations_to_single_round(unreason_ds):
    new_samples = []
    
    for sample in unreason_ds:
        convs = sample['conversations']
        # 按 user -> assistant 成对提取
        for i in range(0, len(convs), 2):
            if i + 1 < len(convs):
                user_turn = convs[i]
                assistant_turn = convs[i + 1]
                # 确保角色正确
                if user_turn['role'] == 'user' and assistant_turn['role'] == 'assistant':
                    new_sample = {
                        'conversations': [user_turn, assistant_turn]
                    }
                    new_samples.append(new_sample)
    return new_samples

# 执行拆分
flat_conversations = split_conversations_to_single_round(unreason_ds)

# 转为 HuggingFace Dataset
new_dataset = Dataset.from_list(flat_conversations)

# 输出查看
print(new_dataset)
print(new_dataset[0])

Dataset({
    features: ['conversations'],
    num_rows: 15
})
{'conversations': [{'content': '问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关', 'role': 'user'}, {'content': '答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。', 'role': 'assistant'}]}


In [74]:
new_dataset[3]

{'conversations': [{'content': '请阅读问题并提供正确的答复。\n问题：女性，50岁，上腹痛3月余，2月前钡透为胃窦后壁溃疡，经抗溃疡药物治疗8周疼痛曾一过性缓解，进一步处理应首选__选项：A. 继续药物治疗 B. 反复查便隐血试验 C. 钡餐检查 D. 胃镜检查 E. 手术治疗 ',
   'role': 'user'},
  {'content': '答案是：D', 'role': 'assistant'}]}

In [75]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

# Apply the formatting function to your dataset using the map method
test_dataset = new_dataset.map(formatting_prompts_func, batched = True,remove_columns=new_dataset.column_names)
test_dataset

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 15
})

In [77]:
test_dataset[0]

{'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关<|im_end|>\n<|im_start|>assistant\n答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。<|im_end|>\n'}

In [79]:
from unsloth.chat_templates import get_chat_template
# 可以利用 get_chat_template 选择喜欢的格式 比如llama
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

# Apply the formatting function to your dataset using the map method
test_dataset = new_dataset.map(formatting_prompts_func, batched = True,remove_columns=new_dataset.column_names)
test_dataset[0]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

{'text': '<|start_header_id|>user<|end_header_id|>\n\n问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。<|eot_id|>'}

In [6]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

from huggingface_hub import notebook_login

notebook_login()

Exception in thread Thread-10 (_readerthread):
Traceback (most recent call last):
  File "e:\Workspace\Anaconda\envs\deep_learning\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "e:\Workspace\Anaconda\envs\deep_learning\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "e:\Workspace\Anaconda\envs\deep_learning\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "e:\Workspace\Anaconda\envs\deep_learning\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 10: invalid start byte


In [7]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
reason_ds = load_dataset("Ronndy/medical_o1_sft_Chinese",cache_dir="./data/reason/",split="train[:10]")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
reason_ds

Dataset({
    features: ['messages'],
    num_rows: 10
})

In [40]:
import json
import random
import re

data_to_shuffle = []

# 3. Process the reasoning dataset (reason_ds)
print(f"Starting to process the reasoning dataset (reason_ds), which has {len(reason_ds)} items...")
for item in reason_ds:
    messages = item.get('messages', [])
    if len(messages) < 3:
        continue

    user_question = messages[1].get('content', '')
    assistant_full_response = messages[2].get('content', '')

    think_match = re.search(r'<think>(.*?)</think>', assistant_full_response, re.DOTALL)
    response_match = re.search(r'<response>(.*?)</response>', assistant_full_response, re.DOTALL)
    cot_content = think_match.group(1).strip() if think_match else ""
    answer_content = response_match.group(1).strip() if response_match else ""

    # Add the processed single dictionary as a "data block"
    data_to_shuffle.append({
        "question": user_question,
        "cot": cot_content,
        "answer": answer_content,
        "category": "reasoning"
    })
print("Reasoning dataset processing complete.")

# 4. Process the non-reasoning dataset (unreason_ds)
print(f"\nStarting to process the non-reasoning dataset (unreason_ds), which has {len(unreason_ds)} items...")
for item in unreason_ds:
    conversations = item.get('conversations', [])
    
    # Create a "block" for the current multi-turn conversation
    conversation_block = []
    
    for i in range(0, len(conversations), 2):
        if i + 1 < len(conversations):
            user_turn = conversations[i]
            assistant_turn = conversations[i+1]
            
            if user_turn.get('role') == 'user' and assistant_turn.get('role') == 'assistant':
                user_question = user_turn.get('content', '')
                assistant_answer = assistant_turn.get('content', '')
                
                # Add the processed turn to the current "block"
                conversation_block.append({
                    "question": user_question,
                    "cot": "", # COT is empty for this category
                    "answer": assistant_answer,
                    "category": "un_reasoning"
                })

    # Add the entire "block" to the list to be shuffled, only if it's not empty
    if conversation_block:
        data_to_shuffle.append(conversation_block)

print("Non-reasoning dataset processing complete.")

# 5. Shuffle the "data blocks"
print(f"\nProcessing finished. Obtained a total of {len(data_to_shuffle)} shuffleable data blocks.")
print("Shuffling the data blocks...")
random.shuffle(data_to_shuffle)
print("Data blocks have been shuffled.")

# 6. Flatten the list and write to a JSON file
final_processed_data = []
for block in data_to_shuffle:
    if isinstance(block, list):
        # If the block is a list (a multi-turn conversation), extend the final list with its items
        final_processed_data.extend(block)
    else:
        # If the block is a dict (a single reasoning entry), append it directly
        final_processed_data.append(block)
        
output_filename = 'processed_dataset_en.json'
print(f"\nWriting {len(final_processed_data)} final records to file: {output_filename} ...")
with open(output_filename, 'w', encoding='utf-8') as f:
    # Use ensure_ascii=False for Chinese characters and indent for readability
    json.dump(final_processed_data, f, ensure_ascii=False, indent=4)

print("\nTask complete!")
print(f"The integrated and corrected dataset has been saved as '{output_filename}'.")

Starting to process the reasoning dataset (reason_ds), which has 10 items...
Reasoning dataset processing complete.

Starting to process the non-reasoning dataset (unreason_ds), which has 10 items...
Non-reasoning dataset processing complete.

Processing finished. Obtained a total of 20 shuffleable data blocks.
Shuffling the data blocks...
Data blocks have been shuffled.

Writing 25 final records to file: processed_dataset_en.json ...

Task complete!
The integrated and corrected dataset has been saved as 'processed_dataset_en.json'.


In [44]:
# 定义输入和输出文件名
input_file = r"F:\workspace\Fine_tuning_all\based_Unsloth\processed_dataset_en.json"
output_file = "./chatml_dataset.json"

# 执行转换
convert_to_chatml(input_file, output_file)

✅ 文件转换成功！
总共处理了 25 条数据。
结果已保存至: ./chatml_dataset.json


In [52]:
import json

def convert_to_chatml(json_file_path):
    """
    读取 JSON 文件并将其转换为 ChatML 格式。
    
    Args:
        json_file_path (str): JSON 文件路径
    
    Returns:
        list: 包含多个字典的列表，每个字典包含 'messages' 键
    """
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    chatml_data = []
    
    for item in data:
        instruction = item["question"]
        cot = item["cot"]
        answer = item["answer"]
        
        # 拼接 output: <think>cot</think> + answer
        output = f"<think>{cot}</think>{answer}"
        
        # 构建 messages 结构
        messages = [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": output}
        ]
        
        chatml_data.append({"conversations": messages})
    
    return chatml_data
result = convert_to_chatml(r"F:\workspace\Fine_tuning_all\based_Unsloth\processed_dataset_en.json")
result[0]

{'conversations': [{'role': 'user',
   'content': '请阅读问题并提供正确的答复。\n问题：女性，50岁，上腹痛3月余，2月前钡透为胃窦后壁溃疡，经抗溃疡药物治疗8周疼痛曾一过性缓解，进一步处理应首选__选项：A. 继续药物治疗 B. 反复查便隐血试验 C. 钡餐检查 D. 胃镜检查 E. 手术治疗 '},
  {'role': 'assistant', 'content': '<think></think>答案是：D'}]}

In [53]:
from datasets import Dataset
result = Dataset.from_list(result)

In [54]:
result

Dataset({
    features: ['conversations'],
    num_rows: 25
})

In [57]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

train_dataset = result.map(formatting_prompts_func, batched = True,remove_columns=result.column_names)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [58]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 25
})

In [60]:
train_dataset[10]

{'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n在足月顺产后，胎儿娩出时张女士阴道出血量约500ml，血液呈鲜红色且很快凝集成血块，但胎盘尚未娩出。这种情况下，出血的最可能原因是什么？<|im_end|>\n<|im_start|>assistant\n<think>哦，张女士在宝宝出生后的出血情况得仔细看看。出血量有500毫升，感觉有点多啊，鲜红色的血液流得快，却很快又凝结成血块，这有点意思。看来血液凝结的能力还不错，应该不是凝血功能有问题吧。\n\n不过，奇怪的是胎盘还没娩出。那出血的问题会不会和胎盘有关系呢？想想看，胎盘要是剥离不全的话，可能会开始出血。不过，血液应该是暗红色而不是鲜红色吧？这不太符合呀。\n\n另外，胎盘滞留确实是个大问题，尤其是在第三产程，不过这通常胎盘剥离的时候开始的，和这种鲜红出血不太对得上。\n\n还有个问题得考虑下，这种鲜红的血容易让人想到动脉的出血，也许是挺快的。再看，看起来胎盘的问题不太说得通，其他可能性呢？比如，子宫的宫缩乏力导致的大出血通常是在胎盘娩出后呀，现在胎盘还没下来呢。\n\n那会不会是生产过程中造成的撕裂或者损伤呢？软产道撕裂可以导致新鲜、鲜红的出血，这种出血的性质倒是和我们现在看到的差不多哦。\n\n嗯，回过头来看，虽然胎盘尚未娩出的确悬在心上，但基于出血的鲜红和快速凝结的特征，再加上一时看不到其他更切合的原因，能解释这鲜红色出血的就剩下软产道损伤了吧。</think>在这种情况下，张女士在胎儿娩出后出现的鲜红色出血，很可能是由于软产道损伤引起的。虽然胎盘尚未娩出通常会关注到胎盘剥离不全或胎盘滞留的问题，但这些情况通常伴随着暗红色出血，而非鲜红色。此外，鲜红色出血快速凝结成血块，也表明她的凝血功能似乎没有问题，因此最合理的推断是生产过程中造成了软产道撕裂，这种撕裂会导致鲜红色的出血。建议尽快由医护人员进行检查和处理，以确保安全。<|im_end|>\n'}